In [567]:
import argparse
import os
import vcf
import subprocess

In [568]:
#coordinates of the 3 genes in 1-based coordinates (because VCF is 1-based)
#protease
pStart = 2253
pEnd = 2549
#reverse-transcriptase
rtStart = 2550
rtEnd = 4229
#integrase
iStart = 4230
iEnd = 5093

In [569]:
#HIVDB protein sequences:
protease_hivdb="PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKMIGGIGGFIKVRQYDQILIEICGHKAIGTVLVGPTPVNIIGRNLLTQIGCTLNF"

rt_hivdb = "PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGKISKI"\
     "GPENPYNTPVFAIKKKDSTKWRKLVDFRELNKRTQDFWEVQLGIPHPAGL"\
     "KKKKSVTVLDVGDAYFSVPLDKDFRKYTAFTIPSINNETPGIRYQYNVLP"\
     "QGWKGSPAIFQSSMTKILEPFRKQNPDIVIYQYMDDLYVGSDLEIGQHRT"\
     "KIEELRQHLLRWGFTTPDKKHQKEPPFLWMGYELHPDKWTVQPIVLPEKD"\
     "SWTVNDIQKLVGKLNWASQIYAGIKVKQLCKLLRGTKALTEVIPLTEEAE"\
     "LELAENREILKEPVHGVYYDPSKDLIAEIQKQGQGQWTYQIYQEPFKNLK"\
     "TGKYARMRGAHTNDVKQLTEAVQKIATESIVIWGKTPKFKLPIQKETWEA"\
     "WWTEYWQATWIPEWEFVNTPPLVKLWYQLEKEPIVGAETFYVDGAANRET"\
     "KLGKAGYVTDRGRQKVVSLTDTTNQKTELQAIHLALQDSGLEVNIVTDSQ"\
     "YALGIIQAQPDKSESELVSQIIEQLIKKEKVYLAWVPAHKGIGGNEQVDK"\
     "LVSAGIRKVL"

integrase_hivdb = "FLDGIDKAQEEHEKYHSNWRAMASDFNLPPVVAKEIVASCDKCQLKGEAM"\
            "HGQVDCSPGIWQLDCTHLEGKIILVAVHVASGYIEAEVIPAETGQETAYF"\
            "LLKLAGRWPVKTIHTDNGSNFTSTTVKAACWWAGIKQEFGIPYNPQSQGV"\
            "VESMNKELKKIIGQVRDQAEHLKTAVQMAVFIHNFKRKGGIGGYSAGERI"\
            "VDIIATDIQTKELQKQITKIQNFRVYYRDSRDPLWKGPAKLLWKGEGAVV"\
            "IQDNSDIKVVPRRKAKIIRDYGKQMAGDDCVASRQDED"

In [570]:
protease_hxb2 = "CCTCAGGTCACTCTTTGGCAACGACCCCTCGTCACAATAAAGATAGGGGGGCAACTAAAGGAAGCTCTATTAGATACAGGAGCAGATGATACAGTATTAGAAGAAATGAGTTTGCCAGGAAGATGGAAACCAAAAATGATAGGGGGAATTGGAGGTTTTATCAAAGTAAGACAGTATGATCAGATACTCATAGAAATCTGTGGACATAAAGCTATAGGTACAGTATTAGTAGGACCTACACCTGTCAACATAATTGGAAGAAATCTGTTGACTCAGATTGGTTGCACTTTAAATTTT"
rt_hxb2 = "CCCATTAGCCCTATTGAGACTGTACCAGTAAAATTAAAGCCAGGAATGGATGGCCCAAAAGTTAAACAATGGCCATTGACAGAAGAAAAAATAAAAGCATTAGTAGAAATTTGTACAGAGATGGAAAAGGAAGGGAAAATTTCAAAAATTGGGCCTGAAAATCCATACAATACTCCAGTATTTGCCATAAAGAAAAAAGACAGTACTAAATGGAGAAAATTAGTAGATTTCAGAGAACTTAATAAGAGAACTCAAGACTTCTGGGAAGTTCAATTAGGAATACCACATCCCGCAGGGTTAAAAAAGAAAAAATCAGTAACAGTACTGGATGTGGGTGATGCATATTTTTCAGTTCCCTTAGATGAAGACTTCAGGAAGTATACTGCATTTACCATACCTAGTATAAACAATGAGACACCAGGGATTAGATATCAGTACAATGTGCTTCCACAGGGATGGAAAGGATCACCAGCAATATTCCAAAGTAGCATGACAAAAATCTTAGAGCCTTTTAGAAAACAAAATCCAGACATAGTTATCTATCAATACATGGATGATTTGTATGTAGGATCTGACTTAGAAATAGGGCAGCATAGAACAAAAATAGAGGAGCTGAGACAACATCTGTTGAGGTGGGGACTTACCACACCAGACAAAAAACATCAGAAAGAACCTCCATTCCTTTGGATGGGTTATGAACTCCATCCTGATAAATGGACAGTACAGCCTATAGTGCTGCCAGAAAAAGACAGCTGGACTGTCAATGACATACAGAAGTTAGTGGGGAAATTGAATTGGGCAAGTCAGATTTACCCAGGGATTAAAGTAAGGCAATTATGTAAACTCCTTAGAGGAACCAAAGCACTAACAGAAGTAATACCACTAACAGAAGAAGCAGAGCTAGAACTGGCAGAAAACAGAGAGATTCTAAAAGAACCAGTACATGGAGTGTATTATGACCCATCAAAAGACTTAATAGCAGAAATACAGAAGCAGGGGCAAGGCCAATGGACATATCAAATTTATCAAGAGCCATTTAAAAATCTGAAAACAGGAAAATATGCAAGAATGAGGGGTGCCCACACTAATGATGTAAAACAATTAACAGAGGCAGTGCAAAAAATAACCACAGAAAGCATAGTAATATGGGGAAAGACTCCTAAATTTAAACTGCCCATACAAAAGGAAACATGGGAAACATGGTGGACAGAGTATTGGCAAGCCACCTGGATTCCTGAGTGGGAGTTTGTTAATACCCCTCCCTTAGTGAAATTATGGTACCAGTTAGAGAAAGAACCCATAGTAGGAGCAGAAACCTTCTATGTAGATGGGGCAGCTAACAGGGAGACTAAATTAGGAAAAGCAGGATATGTTACTAATAGAGGAAGACAAAAAGTTGTCACCCTAACTGACACAACAAATCAGAAGACTGAGTTACAAGCAATTTATCTAGCTTTGCAGGATTCGGGATTAGAAGTAAACATAGTAACAGACTCACAATATGCATTAGGAATCATTCAAGCACAACCAGATCAAAGTGAATCAGAGTTAGTCAATCAAATAATAGAGCAGTTAATAAAAAAGGAAAAGGTCTATCTGGCATGGGTACCAGCACACAAAGGAATTGGAGGAAATGAACAAGTAGATAAATTAGTCAGTGCTGGAATCAGGAAAGTACTA"
integrase_hxb2 = "TTTTTAGATGGAATAGATAAGGCCCAAGATGAACATGAGAAATATCACAGTAATTGGAGAGCAATGGCTAGTGATTTTAACCTGCCACCTGTAGTAGCAAAAGAAATAGTAGCCAGCTGTGATAAATGTCAGCTAAAAGGAGAAGCCATGCATGGACAAGTAGACTGTAGTCCAGGAATATGGCAACTAGATTGTACACATTTAGAAGGAAAAGTTATCCTGGTAGCAGTTCATGTAGCCAGTGGATATATAGAAGCAGAAGTTATTCCAGCAGAAACAGGGCAGGAAACAGCATATTTTCTTTTAAAATTAGCAGGAAGATGGCCAGTAAAAACAATACATACTGACAATGGCAGCAATTTCACCGGTGCTACGGTTAGGGCCGCCTGTTGGTGGGCGGGAATCAAGCAGGAATTTGGAATTCCCTACAATCCCCAAAGTCAAGGAGTAGTAGAATCTATGAATAAAGAATTAAAGAAAATTATAGGACAGGTAAGAGATCAGGCTGAACATCTTAAGACAGCAGTACAAATGGCAGTATTCATCCACAATTTTAAAAGAAAAGGGGGGATTGGGGGGTACAGTGCAGGGGAAAGAATAGTAGACATAATAGCAACAGACATACAAACTAAAGAATTACAAAAACAAATTACAAAAATTCAAAATTTTCGGGTTTATTACAGGGACAGCAGAAATCCACTTTGGAAAGGACCAGCAAAGCTCCTCTGGAAAGGTGAAGGGGCAGTAGTAATACAAGATAATAGTGACATAAAAGTAGTGCCAAGAAGAAAAGCAAAGATCATTAGGGATTATGGAAAACAGATGGCAGGTGATGATTGTGTGGCAAGTAGACAGGATGAGGAT"

In [571]:
geneticCode ={
#Phenylalanine
"TTT": "F", "TTC": "F",
#Leucine
"TTA": "L", "TTG": "L", "CTT": "L", "CTC": "L", "CTA": "L", "CTG": "L",
#Isoleucine
"ATT": "I", "ATC": "I","ATA": "I",
#Methioinine
"ATG": "M",
#Valine
"GTT": "V", "GTC": "V", "GTA": "V", "GTG": "V",
#Serine
"TCT": "S", "TCC": "S", "TCA": "S","TCG": "S","AGT": "S", "AGC": "S",
#Proline
"CCT": "P", "CCC": "P", "CCA": "P","CCG": "P",
#Threonine
"ACT": "T","ACC": "T","ACA": "T","ACG": "T",
#Alanine
"GCT": "A","GCC": "A","GCA": "A","GCG": "A",
#Tyrosine
"TAT": "Y","TAC": "Y",
#Histidine
"CAT": "H","CAC": "H",
#Glutamine
"CAA": "Q","CAG": "Q",
#Asparagine
"AAT": "N","AAC": "N",
#Lysine
"AAA": "K","AAG": "K",
#Aspartic acid
"GAT": "D","GAC": "D",
#Glutamic acid
"GAA": "E","GAG": "E",
#Cysteine
"TGT": "C","TGC": "C",
#Tryptophan
"TGG": "W",
#Arginine
"CGT": "R","CGC": "R","CGA": "R","CGG": "R","AGA": "R","AGG": "R",
#Glycine
"GGT": "G","GGC": "G","GGA": "G","GGG": "G",
#stop
"TAA": "*","TAG": "*","TGA": "*"
}

In [572]:
def getCombinations(base1,  base2, base3,reference):
# returns all possible combinations
    mutations = []
    #print("Bases", base1, base2, base3)
    for i in range (len(base1)):
        for j in range (len(base2)):
            for k in range (len(base3)):
                codonStr = str(base1[i])+str(base2[j])+str(base3[k])
                #print("CODON:",codonStr)
                trans = geneticCode.get(codonStr,"!")
                #print("TRANS:",trans, reference)
                if trans!="!" and trans!=reference:
                    #add as a mutation only if it differs from the reference
                    if trans not in mutations:
                        #print("ALT codon",codonStr)
                        mutations.append(trans)
    return mutations

In [573]:
def getMutations_mult(mut,refG):
    #returns all possible amino acid mutations per position
    #protease
    mutList = []
    ref_gene = list(refG)
    codonct = 1
    #print("LENPROT",len(prot))
    for i in range(0,len(mut),3):
        #print("I",i,prot[i])
        #print("CODONCT ",codonct-1,ref_protease[codonct-1])
        #print("Base:",prot[i],prot[i+1],prot[i+2])
        combinations = getCombinations(mut[i],mut[i+1], mut[i+2],ref_gene[codonct-1])
        if len(combinations)>0:
            # print("MUTATION:", codonct, ref_gene[codonct-1], "comb", combinations)
            mutList.append([codonct,ref_gene[codonct-1],combinations])

        codonct = codonct + 1

    #print("M",mutList)
    return mutList

In [574]:
def createHIVDBRequest(prot,rt,integrase, out):
    cmd = "sierrapy mutations "
    #cmd = ""
    #protease
    for i in prot:
        # print("PROT: ", i)
        for j in i[2]:
            #print("SIERRA: " , i[0],i[1], j)
            mut = "PR:"+i[1]+str(i[0])+j
            #print("SIERRA", mut)
            cmd = cmd + " "+mut
    #RT
    for i in rt:
        #print("PROT: ", i)
        for j in i[2]:
        #print("SIERRA: " , i[0],i[1], j)
            mut = "RT:"+i[1]+str(i[0])+j
            #print("SIERRA", mut)
            cmd = cmd + " "+mut
    #I
    for i in integrase:
        #print("PROT: ", i)
        for j in i[2]:
            #print("SIERRA: " , i[0],i[1], j)
            mut = "RT:"+i[1]+str(i[0])+j
            #print("SIERRA", mut)
            cmd = cmd + " "+mut

    output = subprocess.check_output(cmd, shell=True)
    file = open(out, 'w')
    file.write(output.decode("utf-8"))
    file.close()

In [575]:
vcf_reader = vcf.Reader(open("/Users/kimwilliamelee/Git/hiv/test_out/variants/Brazil/SRR7993842_to_subtype_HXB2/paramgroup_1/variants_last.vcf",'r'))

# maf = open(args.mafFile, 'r')  
file = "/Users/kimwilliamelee/Git/hiv/test_out/phase_two_subtype_alignments/hxb2_subtype02_AG_2.maf"
maf = open(file, "r")
mapping = []

for line in (maf.readlines() [-3:]):
    # print(line, end ='')
    mapping.append(line.split())

print(mapping)

[['s', 'hxb2.K03455.1', '416', '9001', '+', '9719', 'AGATCCTGCATATAAGCAGCTGCTTTTTGCCTGTACTGGGTCTCTCTGGTTAGACCAGATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGGAACCCACTGCTTAAGCCTCAATAAAGCTTGCCTTGAGTGCTTCAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAGATCCCTCAGACCCTTTTAGTCAGTGTGGAAAATCTCTAGCAGTGGCGCCCGAACAGGGACCTGAAAG-----------CGAAAGGGAAA---CCAGAGGAGCTCTCTCGACGCAG-GACTCGGCTTGCTGAAGCGCGCACGGCAAGAGGCGAGGGGCGGCGACTGGTGAGTACGCCAAAAATTTTGACTAGCGGAGGCTAGAAGGAGAGAGATGGGTGCGAGAGCGTCAGTATTAAGCGGGGGAGAATTAGATCGATGGgaaaaaattcggttaaggccagggggaaagaaaaaaTATAAATTAAAACATATAGTATGGGCAAGCAGGGAGCTAGAACGATTCGCAGTTAATCCTGGCCTGTTAGAAACATCAGAAGGCTGTAGACAAATACTGGGACAGCTACAACCATCCCTTCAGACAGGATCAGAAGAACTTAGATCATTATATAATACAGTAGCAACCCTCTATTGTGTGCATCAAAGGATAGAGATAAAAGACACCAAGGAAGCTTTAGACAAGATAGAGGAAGAGCAAAACAAAAGTAAGAAAAAAGCACAGCAAGCAGCAGCTGACACAGGACACAGCAATCAGGTCAGCCAAAATTACCCTATAGTGCAGAACATCCAGGGGCAAATGGTACATCAGGCCATATCACCTAGAACTTTAAATGCATGGGTAAAAGTAGTAGAAGAGAAGGCTTTCAGCCCAGAAGTGATACCCATGTTTTCAGCATTATCAGAAGGAGCCACCCCACAAGATTTAAACACCA

In [576]:
def generate_ref_to_hxb2_dict(maf):
    mutation_mapping = maf.readlines()[-3:]

    hxb2 = mutation_mapping[0].split(" ")
    ref = mutation_mapping[1].split(" ")

    # print(hxb2)
    # print(ref)

    hxb2_pos = int(hxb2[-5])
    ref_pos = int(ref[-5])

    print(hxb2_pos, " : ", ref_pos)

    hxb2_seq = hxb2[-1].strip()
    ref_seq = ref[-1].strip()

    print(hxb2_seq)
    print(ref_seq)

    mut_dict = {}
    for ctr in range(len(ref_seq)):
        if hxb2_pos > int(hxb2[-2]):
            break

        if hxb2_seq[ctr] != ref_seq[ctr]:
            mut_dict[hxb2_pos, ref_seq[ctr]] = hxb2_seq[ctr]
            
        hxb2_pos += 1
        ref_pos += 1
        ctr += 1

    return mut_dict

In [577]:
maf = "/Users/kimwilliamelee/Git/hiv/test_out/phase_two_subtype_alignments/hxb2_subtype02_AG_2.maf"

ref_to_hxb2_mut_dict = {}
ref_to_hxb2_mut_dict = generate_ref_to_hxb2_dict(open(maf, 'r'))

[print(key,':',value) for key, value in ref_to_hxb2_mut_dict.items() if key[0] >= pStart and key[0]<pEnd]

# (2684, 'T') : [2268, 'C']

416  :  0
AGATCCTGCATATAAGCAGCTGCTTTTTGCCTGTACTGGGTCTCTCTGGTTAGACCAGATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGGAACCCACTGCTTAAGCCTCAATAAAGCTTGCCTTGAGTGCTTCAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAGATCCCTCAGACCCTTTTAGTCAGTGTGGAAAATCTCTAGCAGTGGCGCCCGAACAGGGACCTGAAAG-----------CGAAAGGGAAA---CCAGAGGAGCTCTCTCGACGCAG-GACTCGGCTTGCTGAAGCGCGCACGGCAAGAGGCGAGGGGCGGCGACTGGTGAGTACGCCAAAAATTTTGACTAGCGGAGGCTAGAAGGAGAGAGATGGGTGCGAGAGCGTCAGTATTAAGCGGGGGAGAATTAGATCGATGGgaaaaaattcggttaaggccagggggaaagaaaaaaTATAAATTAAAACATATAGTATGGGCAAGCAGGGAGCTAGAACGATTCGCAGTTAATCCTGGCCTGTTAGAAACATCAGAAGGCTGTAGACAAATACTGGGACAGCTACAACCATCCCTTCAGACAGGATCAGAAGAACTTAGATCATTATATAATACAGTAGCAACCCTCTATTGTGTGCATCAAAGGATAGAGATAAAAGACACCAAGGAAGCTTTAGACAAGATAGAGGAAGAGCAAAACAAAAGTAAGAAAAAAGCACAGCAAGCAGCAGCTGACACAGGACACAGCAATCAGGTCAGCCAAAATTACCCTATAGTGCAGAACATCCAGGGGCAAATGGTACATCAGGCCATATCACCTAGAACTTTAAATGCATGGGTAAAAGTAGTAGAAGAGAAGGCTTTCAGCCCAGAAGTGATACCCATGTTTTCAGCATTATCAGAAGGAGCCACCCCACAAGATTTAAACACCATGCTAAACACAGTGGGGGGACATCAAGCAGCCATGCAAATGTT

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [578]:
# protease_seq = list(protease_hxb2)
# rt_seq = list(rt_hxb2)
# i_seq = list(integrase_hxb2)

# for record in vcf_reader:
#     #print(record)
#     if record.POS >=pStart and record.POS <pEnd: #protease
#         alt = record.ALT
#         temp = []
#         for i in range(len(alt[0])):
#             base = str(alt[0])[i]
#             if base not in temp:
#                 temp.append([record.POS, base, "new"])
#         protease_seq[record.POS-2253] = temp
#     elif record.POS >=rtStart and record.POS < rtEnd: #3869: #reverse transcriptase
#         alt = record.ALT
#         temp = []
#         for i in range(len(alt[0])):
#             base = str(alt[0])[i]
#             if base not in temp:
#                 temp.append(base)
#         rt_seq[record.POS-2550] = temp
#     elif record.POS >=iStart and record.POS <=iEnd: #5096 minus stop codon #integrase
#         alt = record.ALT
#         temp = []
#         for i in range(len(alt[0])):
#             base = str(alt[0])[i]
#             if base not in temp:
#                 temp.append(base)
#         i_seq[record.POS-4230] = temp

# print(protease_seq)
# print(rt_seq)
# print(i_seq)

In [579]:
protease_seq = list(protease_hxb2)
rt_seq = list(rt_hxb2)
i_seq = list(integrase_hxb2)

for record in vcf_reader:
    #print(record)
    if record.POS >=pStart and record.POS <pEnd: #protease
        alt = record.ALT
        temp = []
        for i in range(len(alt[0])):
            base = str(alt[0])[i]
            if base not in temp:
                # print(record.POS, base, "old")
                if (record.POS, base) in ref_to_hxb2_mut_dict:
                    base = ref_to_hxb2_mut_dict[(record.POS, base)]
                    # print(record.POS, str(alt[0])[i], base, "new")
                temp.append(base)
        protease_seq[record.POS-2253] = temp
    elif record.POS >=rtStart and record.POS < rtEnd: #3869: #reverse transcriptase
        alt = record.ALT
        temp = []
        for i in range(len(alt[0])):
            base = str(alt[0])[i]
            if base not in temp:
                # print(record.POS, base, "old")
                if (record.POS, base) in ref_to_hxb2_mut_dict:
                    base = ref_to_hxb2_mut_dict[(record.POS, base)]
                    # print(record.POS, str(alt[0])[i], base, "new")
                temp.append(base)
        rt_seq[record.POS-2550] = temp
    elif record.POS >=iStart and record.POS <=iEnd: #5096 minus stop codon #integrase
        alt = record.ALT
        temp = []
        for i in range(len(alt[0])):
            base = str(alt[0])[i]
            if base not in temp:
                # print(record.POS, base, "old")
                if (record.POS, base) in ref_to_hxb2_mut_dict:
                    base = ref_to_hxb2_mut_dict[(record.POS, base)]
                    # print(record.POS, str(alt[0])[i], base, "new")
                temp.append(base)
        i_seq[record.POS-4230] = temp

print(protease_seq)
print(rt_seq)
print(i_seq)

['C', 'C', ['G'], 'C', 'A', ['A'], ['A'], ['G'], 'C', ['T'], 'C', 'T', 'C', ['C'], ['G'], ['C'], ['T'], ['T'], ['T'], ['T'], ['T'], 'C', ['T'], ['G'], 'C', 'C', 'C', ['T'], 'T', ['A'], 'G', ['C'], 'C', ['T'], 'C', ['G'], ['G'], 'T', 'A', ['C'], 'A', 'G', ['G'], 'T', 'A', 'G', 'G', 'G', 'G', 'G', ['A'], 'C', 'A', ['G'], 'C', 'T', ['C'], ['T'], ['G'], ['A'], 'G', ['C'], ['G'], 'G', 'C', ['C'], ['A'], ['A'], ['T'], ['G'], ['G'], ['T'], ['T'], ['T'], ['G'], ['T'], 'C', ['T'], ['T'], 'G', ['G'], 'G', ['T'], ['T'], ['T'], ['G'], ['G'], ['T'], ['T'], ['G'], ['T'], ['A'], ['T'], 'G', ['G'], ['C'], ['T'], ['G'], ['C'], 'G', 'A', ['C'], 'G', 'A', ['T'], 'A', 'T', ['A'], ['T'], ['A'], 'T', 'T', 'T', ['A'], ['A'], 'C', 'A', 'G', 'G', 'A', ['C'], ['A'], ['G'], ['C'], ['C'], ['T'], ['T'], ['T'], ['T'], ['A'], ['T'], ['T'], ['T'], ['C'], ['T'], ['G'], ['C'], ['T'], ['T'], ['C'], ['T'], ['T'], ['T'], ['T'], ['T'], ['T'], ['T'], ['T'], ['G'], ['G'], ['T'], ['T'], ['T'], ['T'], ['T'], ['G'], ['G'], ['G'

In [580]:
protease_mut = getMutations_mult(protease_seq,protease_hivdb)
rt_mut = getMutations_mult(rt_seq,rt_hivdb)
int_mut = getMutations_mult(i_seq,integrase_hivdb)

# createHIVDBRequest(protease_mut, rt_mut,int_mut,args.outFile)
outfile = "/Users/kimwilliamelee/Git/hiv/test_out/variants/Brazil/SRR7993842_to_subtype_HXB2/paramgroup_1/output.json"
createHIVDBRequest(protease_mut, rt_mut, int_mut, outfile)

In [581]:
from tabulate import tabulate
import json

data = json.load(open(outfile))
vResults = data['validationResults']
drugResistance = data['drugResistance']

dbResults = ""

# Printing the validation results
# print("Validation Results")
for v in vResults:
    dbResults += "\n" + v['level'] + ": " + v['message'] + "\n"

# Printing drug resistance results
for d in drugResistance:
    dbResults += "\n\nDrug resistance interpretation: " + d['gene']['name']
    dbResults += "\nVersion: " + d['version']['text'] + "("  + d['version']['publishDate'] + ")"

    drugClass = []
    drug = []
    score = []
    partialScores = []
    text = []
    ctr = 0

    # Store the relevant information into a new dictionary
    for d2 in d['drugScores']:
        drugClass.append(d2['drugClass']['name'])
        drug.append(d2['drug']['displayAbbr'])
        score.append(d2['score'])
        partialScores.append(d2['partialScores'])
        text.append(d2['text'])
        ctr+=1

    results = {
        'drugClass' : drugClass,
        'drug' : drug,
        'score' : score,
        'text' : text
    }

    # Display Drug Resistance Results Table
    #display(pd.DataFrame(results))
    dbResults += "\n"
    dbResults += tabulate(results, headers = 'keys', tablefmt = 'psql')
    dbResults += "\n"

    # Printing partial scores, mutations, and comments
    for p in partialScores:
        if(p):
            mutations = p[0]['mutations']

            for m in mutations:
                dbResults += "\n"
                dbResults += "Mutation: " + m['text'] + "\n"
                dbResults += "Type: " + m['primaryType'] + "\n"
                dbResults += "Comments:\n"

                comments = m['comments']
                for c in comments:
                    dbResults += c['text'] + "\n"

print(dbResults)

# f = open(args.outFile, 'w')
# f.write(dbResults)
# f.close()
#print (output)











Drug resistance interpretation: PR
Version: HIVDB_9.4.1(2023-06-11)
+-------------+--------+---------+-------------------------+
| drugClass   | drug   |   score | text                    |
|-------------+--------+---------+-------------------------|
| PI          | ATV/r  |      15 | Low-Level Resistance    |
| PI          | DRV/r  |       0 | Susceptible             |
| PI          | FPV/r  |      15 | Low-Level Resistance    |
| PI          | IDV/r  |      30 | Intermediate Resistance |
| PI          | LPV/r  |      30 | Intermediate Resistance |
| PI          | NFV    |      30 | Intermediate Resistance |
| PI          | SQV/r  |      15 | Low-Level Resistance    |
| PI          | TPV/r  |       0 | Susceptible             |
+-------------+--------+---------+-------------------------+

Mutation: V82A
Type: Major
Comments:
V82A is a non-polymorphic mutation selected primarily by IDV and LPV. It is associated with reduced susceptibility to LPV and to a lesser extent ATV. It 